# Segmenting and Clustering Neighborhoods in Toronto
## Activty 2

### Made by: Holzel, Gabriela

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe.

In [14]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [15]:
!pip install BeautifulSoup4
!pip install requests

In [16]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(source.text, 'lxml')

table = soup.find('table')
table_rows = table.tbody.find_all('tr')

In [72]:
Postcode = []
Borough = []
Neighbourhood = []

for tr in table_rows:
    td = tr.find_all("td")
    for item in td:
        if "Not assigned" not in item.text:
            Postcode.append(item.text.strip()[0:3])
            if item.text.strip().find('(') != -1:
                Borough.append(item.text.strip()[3:item.text.strip().find('(')])
                Neighbourhood.append(item.text.strip()[item.text.strip().find('(')+1:item.text.strip().find(')')].replace(" / ",",").replace("/",","))
            else:
                Borough.append(item.text.strip()[3:len(item.text.strip())])
                Neighbourhood.append(item.text.strip()[3:len(item.text.strip())])
canada_dict = {'Postcode':Postcode, 'Borough':Borough, 'Neighbourhood':Neighbourhood}
CA = pd.DataFrame.from_dict(canada_dict)
CA.to_csv('toronto_part1.csv')
CA.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


We want to see the shape of the dataframe

In [78]:
print('The shape is: ', CA.shape)
print('The columns names are: ', CA.columns)

The shape is:  (103, 3)
The columns names are:  Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')


Now, we install geocoder.

In [19]:
!pip install geocoder

In [21]:
import geocoder # import geocoder


def LL(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        print(lat_lng_coords)
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return (latitude, longitude)

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, we get the csv file that has the geographical coordinates of each postal code from the following link:

In [79]:
PC = pd.read_csv("https://cocl.us/Geospatial_data")
PC.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [80]:
print('The shape is: ', PC.shape)
print('The columns names are: ', PC.columns)

The shape is:  (103, 3)
The columns names are:  Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')


Now we merge both dataframes: CA and PC.

In [95]:
CA_new = pd.merge(CA, PC, how='left', left_on = 'Postcode', right_on = 'Postal Code')
CA_new.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


Now, we remove the "Postal Code" column:

In [96]:
CA_new.drop("Postal Code", axis=1, inplace=True)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
